In [1]:
import os
import re
import time
import pandas as pd
import numpy as np
from datetime import datetime, date

In [53]:
# funcs

def get_team_results(
                    h_team_name: str,
                    a_team_name: str,
                    matches_before: pd.DataFrame,
                    ) -> tuple[int, int, int, int] :
    """
    отримання результатів команд, останнього матчу
    повертає список з 4 чисел, різниці забитих мячів
      - дом. команди в першому таймі,
      - дом. команди в повному матчі,
      - гост. команди в першому таймі,
      - гост. команди в повному матчі,
    """
    h_team_match = matches_before[                            
                        (matches_before["HomeTeam"] == h_team_name) | 
                        (matches_before["AwayTeam"] == h_team_name)
                    ].tail(1) 
    
    a_team_match = matches_before[
                        (matches_before["HomeTeam"] == a_team_name) |
                        (matches_before["AwayTeam"] == a_team_name)
                    ].tail(1) 
    
    print()
    print(h_team_match)
    print()
    print(a_team_match)
    print()

    h_first = (
                    h_team_match.at[h_team_match.index[0], "HTHG"] - 
                    h_team_match.at[h_team_match.index[0], "HTAG"]
                )


    h_full = (
                    h_team_match.at[h_team_match.index[0], "FTHG"] - 
                    h_team_match.at[h_team_match.index[0], "FTAG"]
                )

    h_first = h_first if h_team_name in h_team_match["HomeTeam"].values else -h_first
    h_full = h_full if h_team_name in h_team_match["HomeTeam"].values else -h_full


    a_first = (
                    a_team_match.at[a_team_match.index[0], "HTHG"] - 
                    a_team_match.at[a_team_match.index[0], "HTAG"]
                )

    a_full = (
                    a_team_match.at[a_team_match.index[0], "FTHG"] - 
                    a_team_match.at[a_team_match.index[0], "FTAG"]
                )
    a_first = a_first if a_team_name in a_team_match["HomeTeam"].values else -a_first
    a_full = a_full if a_team_name in a_team_match["HomeTeam"].values else -a_full
    
    return (
        h_first,
        h_full,
        a_first,
        a_full,
    )

In [103]:
# конвертування строчки дати в формат дати 
d, m, y = [int(x) for x in df.at[0, "Date"].split("/")]
a = date(y, m, d)
a
# date(2023, 5, 9).isoweekday() == 2  # первірка дати на вівторок

datetime.date(2021, 8, 13)

In [54]:
df = pd.read_csv("./data/eo_21_22.csv")
df.loc[:, "Date"] =  [date.fromisoformat("-".join(x.split("/")[::-1]))  for x in df.loc[:, "Date"].values]

df

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2021-08-13,20:00,Brentford,Arsenal,2,0,H,1,0,...,1.62,0.50,1.75,2.05,1.81,2.13,2.05,2.17,1.80,2.09
1,E0,2021-08-14,12:30,Man United,Leeds,5,1,H,1,0,...,2.25,-1.00,2.05,1.75,2.17,1.77,2.19,1.93,2.10,1.79
2,E0,2021-08-14,15:00,Burnley,Brighton,1,2,A,1,0,...,1.62,0.25,1.79,2.15,1.81,2.14,1.82,2.19,1.79,2.12
3,E0,2021-08-14,15:00,Chelsea,Crystal Palace,3,0,H,2,0,...,1.94,-1.50,2.05,1.75,2.12,1.81,2.16,1.93,2.06,1.82
4,E0,2021-08-14,15:00,Everton,Southampton,3,1,H,0,1,...,1.67,-0.50,2.05,1.88,2.05,1.88,2.08,1.90,2.03,1.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2022-05-22,16:00,Crystal Palace,Man United,1,0,H,1,0,...,2.04,0.25,1.68,2.15,1.74,2.23,1.88,2.25,1.74,2.16
376,E0,2022-05-22,16:00,Leicester,Southampton,4,1,H,0,0,...,2.63,-0.75,1.83,2.07,1.88,2.03,1.94,2.26,1.87,2.01
377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,1,...,3.28,-2.50,2.02,1.77,2.06,1.83,2.19,1.99,2.07,1.80
378,E0,2022-05-22,16:00,Man City,Aston Villa,3,2,H,0,1,...,3.36,-2.25,2.06,1.84,2.05,1.86,2.09,2.03,2.01,1.87


In [55]:
df_data = df.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HTHG", "HTAG"]]
df_data

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG
0,2021-08-13,Brentford,Arsenal,2,0,1,0
1,2021-08-14,Man United,Leeds,5,1,1,0
2,2021-08-14,Burnley,Brighton,1,2,1,0
3,2021-08-14,Chelsea,Crystal Palace,3,0,2,0
4,2021-08-14,Everton,Southampton,3,1,0,1
...,...,...,...,...,...,...,...
375,2022-05-22,Crystal Palace,Man United,1,0,1,0
376,2022-05-22,Leicester,Southampton,4,1,0,0
377,2022-05-22,Liverpool,Wolves,3,1,1,1
378,2022-05-22,Man City,Aston Villa,3,2,0,1


In [57]:
for ind in df_data.index[14:]:
    rez_df = pd.DataFrame(data=[], columns=[
        "Date",
        "HomeTeam",
        "AwayTeam",
        
    ])
    current_match = df_data.loc[ind]
    matches_before = df_data.loc[:ind-1]
    h_team_name = current_match["HomeTeam"]
    a_team_name = current_match["AwayTeam"]
    if ( 
        (h_team_name in matches_before["HomeTeam"].values or
                 h_team_name in matches_before["AwayTeam"].values) and 
        (a_team_name in matches_before["HomeTeam"].values or
                 a_team_name in matches_before["AwayTeam"].values)  
    ):
        h_first, h_full, a_first, a_full = get_team_results(
                                                     h_team_name,
                                                     a_team_name,
                                                    matches_before
                                                    )
        
        print(h_team_name, a_team_name)
        print()
        print(matches_before.tail(12))
        print()
        
        print(h_full, h_first)
        
        print()
        print(a_full, a_first)
        
        print([
            *current_match.values,
            h_first,
            h_full,
            a_first,
            a_full
        ])
        
        raise


         Date   HomeTeam  AwayTeam  FTHG  FTAG  HTHG  HTAG
9  2021-08-15  Tottenham  Man City     1     0     0     0

         Date HomeTeam   AwayTeam  FTHG  FTAG  HTHG  HTAG
7  2021-08-14  Norwich  Liverpool     0     3     0     1

Man City Norwich

          Date        HomeTeam        AwayTeam  FTHG  FTAG  HTHG  HTAG
2   2021-08-14         Burnley        Brighton     1     2     1     0
3   2021-08-14         Chelsea  Crystal Palace     3     0     2     0
4   2021-08-14         Everton     Southampton     3     1     0     1
5   2021-08-14       Leicester          Wolves     1     0     1     0
6   2021-08-14         Watford     Aston Villa     3     2     2     0
7   2021-08-14         Norwich       Liverpool     0     3     0     1
8   2021-08-15       Newcastle        West Ham     2     4     2     1
9   2021-08-15       Tottenham        Man City     1     0     0     0
10  2021-08-21       Liverpool         Burnley     2     0     1     0
11  2021-08-21     Aston Villa     

RuntimeError: No active exception to reraise

In [183]:
df_data.loc[:2-1]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG
0,2021-08-13,Brentford,Arsenal,2,0,1,0
1,2021-08-14,Man United,Leeds,5,1,1,0
